Dataset: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia

# Import Libraries

In [5]:
from tensorflow import keras
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import tensorflow

In [2]:
IMAGE_SIZE=[224,224,3]

In [3]:
train_path='/chest_xray/train'
valid_path='/chest_xray/val'

In [4]:
vgg=VGG16(input_shape=IMAGE_SIZE,weights='imagenet',include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [6]:
for layer in vgg.layers:
  layer.traiable=False

In [7]:
folders=glob('/chest_xray/train/*')

In [8]:
x=Flatten()(vgg.output)

In [9]:
prediction=Dense(len(folders),activation='softmax')(x)

In [10]:
model=Model(inputs=vgg.input,outputs=prediction)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

# Train Model

In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=False)

In [14]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [15]:
training_set=train_datagen.flow_from_directory('chest_xray/train',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')

Found 5216 images belonging to 2 classes.


In [16]:
test_set=test_datagen.flow_from_directory('chest_xray/test',
                                          target_size=(224,224),
                                          batch_size=32,
                                          class_mode='categorical')

Found 624 images belonging to 2 classes.


In [ ]:
r=model.fit(
    training_set,
    validation_data=test_set,
    epochs=10,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

Plot accuracy

In [ ]:
print(r.history.keys())
# summarize history for accuracy
plt.plot(r.history['accuracy'])
plt.plot(r.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(r.history['loss'])
plt.plot(r.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Save model

In [35]:
model.save('PneumoniaV1Model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Load model and predict

In [1]:
from tensorflow.keras.models import load_model

In [2]:
model2=load_model('PneumoniaV1Model.h5')

In [3]:
model2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
img_path = 'image.jpeg'
img = image.load_img(img_path, target_size=(224, 224))  # Adjust target_size as needed
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = preprocess_input(img_array)  # Preprocess input (e.g., normalize)

# Make predictions
predictions = model2.predict(img_array)

1/1 [==============================] - 0s 24ms/step


In [14]:
threshold = 0.5

if predictions[0][1] > threshold:
    print("The individual has the disease.")
else:
    print("The individual does not have the disease.")


The individual has the disease.
